In [178]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as ac
from selenium.webdriver.common.alert import Alert
import win32com.client as win32
import pandas as pd
import time


servico = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument(r'user-data-dir=C:\Users\Willyan\AppData\Local\Google\Chrome\User Data\Profile Selenium')
nav = webdriver.Chrome(service=servico, options=options)
nav.maximize_window() 

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Argila verde,pasta,1,25
1,Argila vermelha,pasta,1,25
2,Argila preta,pasta,1,25
3,Dolomita,pasta,1,25
4,Essência de Camomila,POP,17,25
5,Essência de Bamboo Chamomile,Vela,14,25
6,Essência de Maracujá doce,POP,14,25
7,Base Glicerinada V&G,Petróleo,18,29


Definição das funções de busca no google e no buscape

In [179]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # entrar no google
    nav.get("https://www.google.com/")
    
    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    

    # pesquisar o nome do produto no google
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    # clicar na aba shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar a lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = [] # lista que a função vai me dar como resposta
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()

        # verificacao do nome - se no nome tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto: # verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'QIrs8').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue

            
    return lista_ofertas
    
    

def busca_imperiobanho(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    #tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    nav.get("https://www.imperiodobanho.com.br/")
    
    # pesquisar pelo produto no buscape
    nav.find_element(By.CLASS_NAME, 'input-search').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(5)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'product variant')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'price-off').text
            nome = resultado.get_attribute('product-name')
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas

Contrução da nossa lista de ofertas encontradas

In [180]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_imperiobanho = busca_imperiobanho(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_imperiobanho:
        tabela_imperiobanho = pd.DataFrame(lista_ofertas_imperiobanho, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_imperiobanho)
    else:
        tabela_imperiobanho = None

display(tabela_ofertas)

C:\Users\Willyan\AppData\Local\Temp\ipykernel_9080\3911506987.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\Willyan\AppData\Local\Temp\ipykernel_9080\3911506987.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\Willyan\AppData\Local\Temp\ipykernel_9080\3911506987.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\Willyan\AppData\Local\Temp\ipykernel_9080\3911506987.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

,produto,preco,link
0,argila facial 1kg (ervas raizes) [escolher: ve...,10.00,https://www.google.com/url?url=https://shopee....
1,argila verde ( a granel ),1.20,https://www.google.com/url?url=https://www.gra...
2,argila verde 100% natural - 1 kg mapric,19.50,https://www.google.com/url?url=https://produto...
3,argila verde reduz oleosidade (skin care) torr...,7.99,https://www.google.com/url?url=https://shopee....
4,argila verde 100% natural - 1 kg mapric,18.00,https://www.google.com/url?url=https://produto...
...,...,...,...
1,base glicerinada vegetal v&g - 1kg v & g1kg,24.50,https://www.google.com/url?url=https://shopee....
2,base glicerinada v&g base para sabonete artesa...,24.90,https://www.google.com/url?url=https://shopee....
3,base de glicerinada v&g transparente para sabo...,23.00,https://www.google.com/url?url=https://shopee....
4,base glicerinada v&g para sabonetes 1kg v & g,21.50,https://www.google.com/url?url=https://shopee....


Exportar a base de ofertas para Excel

In [181]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)


Enviando o e-mail

In [182]:
# enviar por e-mail o resultado da tabela

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'willyangomes86@gmail.com'
    mail.CC = 'tracosdanatureza@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att., Willyan Gomes</p>
    """
    
    mail.Send()

nav.quit()  

In [183]:
#nav.get("https://www.google.com/")
#produto = 'Essência de Camomila 100ml'
#nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
#nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
#for item in elementos:
    #if "Shopping" in item.text:
        #item.click()
        #break
        
#elemento_link = nav.find_element(By.CLASS_NAME, 'aULzUe')
#elemento_pai = elemento_link.find_element(By.XPATH, '..')
#link = elemento_pai.get_attribute('href')
#print(link)
#lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')   
#lista_ofertas = [] # lista que a função vai me dar como resposta
#for resultado in lista_resultados:
    #preco = resultado.find_element(By.CLASS_NAME, 'QIrs8').text
   # preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
   # print(preco)
    
#lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
#lista_ofertas = [] # lista que a função vai me dar como resposta
#for resultado in lista_resultados:
   # nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
    #nome = nome.lower()
    #print(nome)